In [12]:
# Test Model
import os 
import re
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model = joblib.load("..//Model//pca8_poly4_linear.pkl")
files = os.listdir('..//Data//ProcessedData')
files.sort(key=lambda x: int(x[1:x.index('_')]))

In [29]:

metrics = {
    'Device_ID': [], 
    'ERROR_SUM': [],
    'MAE': [], 
    'MSE': [], 
    'RMSE': [], 
    'R2': [], 
    }

selected_features = model.feature_names_in_.tolist()
to_predict_features = 'Avg_Power(mW)'

for file in files:
    data_df = pd.read_csv(f'..//Data//ProcessedData//{file}')
    X = data_df[selected_features]
    y = data_df[to_predict_features].to_frame()
    
    # Make predictions
    y_pred = model.predict(X)
    y_pred = np.clip(np.round(y_pred, 2), 0, None)
    y_pred_df = pd.DataFrame(y_pred, columns=y.columns)
    
    # Calculate metrics
    error = abs(y - y_pred_df)
    mse_value = mean_squared_error(y, y_pred)
    mae_value = mean_absolute_error(y, y_pred)
    rmse_value = np.sqrt(mse_value)
    r2_value = r2_score(y, y_pred)
    error_sum_value = error.to_numpy().sum()

    # Append to respective lists
    metrics['Device_ID'].append(int(re.search(r'\d+', file).group()))
    metrics['ERROR_SUM'].append(error_sum_value)
    metrics['MAE'].append(mae_value)
    metrics['MSE'].append(mse_value)
    metrics['RMSE'].append(rmse_value)
    metrics['R2'].append(r2_value)

# Create a DataFrame from the metrics dictionary
metrics_df = pd.DataFrame(metrics)

,Device_ID,ERROR_SUM,MAE,MSE,RMSE,R2
0,4,48957.54,8.504002,640.981091,25.317604,0.997397
1,17,66701.13,6.790993,562.698833,23.721274,0.996571
2,16,42534.73,5.092150,452.015512,21.260656,0.998363
3,10,52222.80,5.640220,437.650995,20.920110,0.997022
4,11,21698.80,3.752819,422.356013,20.551302,0.994071
5,5,40711.29,7.286789,419.266902,20.476008,0.998411
6,8,46451.06,3.861268,411.107383,20.275783,0.997009
7,9,39465.37,4.334472,383.354470,19.579440,0.996474
8,7,30829.59,6.986084,356.860899,18.890762,0.998953
9,6,37424.19,7.038591,351.858393,18.757889,0.998916


In [ ]:

metrics_df_sorted = metrics_df.sort_values(by='RMSE', ascending=False, ignore_index=True)
metrics_df_sorted